In [1]:
import pandas as pd
from multicall.call import Call
from multicall.fetch_multicall_across_blocks import fetch_save_and_return, CACHE_PATH
from multicall.constants import W3
from multicall.cache import create_db, fetch_all_data
import sqlite3


def identity_function(x):
    return x


uniswap_v3_usdc_weth_pool = "0x88e6A0c2dDD26FEEb64F039a2c41296FcB3f5640"
weth = "0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2"
usdc = "0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48"

weth_bal = Call(
    weth,
    "balanceOf(address)(uint256)",
    (uniswap_v3_usdc_weth_pool),
    "weth_bal",
    identity_function,
)
import random

block_to_check = random.randint(10_000_000, 20_000_000)

# Call In Jupyter

In [2]:
from multicall.utils import time_function


@time_function
def a():
    print(weth_bal(W3, block_to_check))

In [3]:
# first
a()

{'weth_bal': 70336717921400434806232}
a executed in 0.646432 seconds.


In [4]:
# second
a()

{'weth_bal': 70336717921400434806232}
a executed in 0.000750 seconds.


# Test Mulitcall In jupyter

In [5]:
usdc_bal = Call(
    usdc,
    "balanceOf(address)(uint256)",
    (uniswap_v3_usdc_weth_pool),
    "usdc_bal",
    identity_function,
)

from multicall.multicall import Multicall


m = Multicall([weth_bal, usdc_bal])


@time_function
def b():
    print(m(W3, block_to_check))

In [6]:
# first
b()

get_data_from_disk executed in 0.003255 seconds.


get_data_from_disk executed in 0.002799 seconds.
{'weth_bal': 70336717921400434806232, 'usdc_bal': 68976422918043, 'block': 18365053}
b executed in 0.441685 seconds.


In [7]:
# second
b()

get_data_from_disk executed in 0.003624 seconds.
{'weth_bal': 70336717921400434806232, 'usdc_bal': 68976422918043, 'block': 18365053}
b executed in 0.004792 seconds.


# Test Fetch and Save


In [8]:
# first
fetch_save_and_return([usdc_bal, weth_bal], [block_to_check - 1, block_to_check, block_to_check + 1], W3)

get_data_from_disk executed in 0.003106 seconds.
found_df.shape=(2, 9)      not_found_df.shape=(4, 9) 

Some data not found, making 2 external calls at a rate of 10 call /second 

get_data_from_disk executed in 0.002772 seconds.
After fetching missing data found_df.shape=(6, 9)      not_found_df.shape=(0, 9) 

fetch_save_and_return executed in 0.319929 seconds.


,usdc_bal,weth_bal,block
0,69238515114651,70172290485495545548788,18365052
1,68976422918043,70336717921400434806232,18365053
2,68976422918043,70336717921400434806232,18365054


In [9]:
# second
fetch_save_and_return([usdc_bal, weth_bal], [block_to_check - 1, block_to_check, block_to_check + 1], W3)

get_data_from_disk executed in 0.003315 seconds.
found_df.shape=(6, 9)      not_found_df.shape=(0, 9) 

all data on disk, no external calls needed!!!
fetch_save_and_return executed in 0.003933 seconds.


,usdc_bal,weth_bal,block
0,69238515114651,70172290485495545548788,18365052
1,68976422918043,70336717921400434806232,18365053
2,68976422918043,70336717921400434806232,18365054


In [10]:
def norm_18(x):
    return x / 1e18


def norm_6(x):
    return x / 6


weth_bal_different_handle_function = Call(
    weth,
    "balanceOf(address)(uint256)",
    (uniswap_v3_usdc_weth_pool),
    "weth_bal",
    norm_18,
)


usdc_bal_different_handle_function = Call(
    usdc,
    "balanceOf(address)(uint256)",
    (uniswap_v3_usdc_weth_pool),
    "usdc_bal",
    norm_6,
)

In [11]:
# third
fetch_save_and_return(
    [usdc_bal_different_handle_function, weth_bal], [block_to_check - 1, block_to_check, block_to_check + 1], W3
)

get_data_from_disk executed in 0.002924 seconds.
found_df.shape=(6, 9)      not_found_df.shape=(0, 9) 

all data on disk, no external calls needed!!!
fetch_save_and_return executed in 0.003433 seconds.


,usdc_bal,weth_bal,block
0,1.153975e+13,70172290485495545548788,18365052
1,1.149607e+13,70336717921400434806232,18365053
2,1.149607e+13,70336717921400434806232,18365054


# Test with size,

- Get the USDC, and WETH balance for 1_000 addresses
- also get the get the addr.balanceOf(addr) for all the addresses

- 1k blocks and 1k function calls, 1M points of state

In [12]:
1000 * 1000

1000000

In [13]:
df = pd.read_csv("../many_weth_transfers_more.csv")
all_tos = df["from"].unique()[:10_000]
len(all_tos)

10000

In [16]:
calls = []
for person in all_tos:
    calls.append(Call(usdc, "balanceOf(address)(uint256)", (person), f"{person}_USDC", norm_6))
    calls.append(Call(weth, "balanceOf(address)(uint256)", (person), f"{person}_WETH", norm_18))

blocks = [18_000_000 + i * 100 for i in range(10)]
len(calls), len(blocks) # wide and narrow only 10 blocks, lots of state

(20000, 10)

2k calls, 1k blocks, 2M points of sta

In [17]:
# first
fetch_save_and_return(calls, blocks, W3)

get_data_from_disk executed in 6.774093 seconds.
found_df.shape=(20000, 9)      not_found_df.shape=(179304, 9) 

Some data not found, making 10 external calls at a rate of 10 call /second 



TimeoutError: 

In [ ]:
# second
fetch_save_and_return(calls, blocks, W3)

get_data_from_disk executed in 2.678953 seconds.
found_df.shape=(200000, 9)      not_found_df.shape=(0, 9) 

all data on disk, no external calls needed!!!
fetch_save_and_return executed in 5.020453 seconds.


,0x7a250d5630B4cF539739dF2C5dAcb4c659F2488D_USDC,0x7a250d5630B4cF539739dF2C5dAcb4c659F2488D_WETH,0xAF0B0000f0210D0f421F0009C72406703B50506B_USDC,0xAF0B0000f0210D0f421F0009C72406703B50506B_WETH,0x19c10E1F20df3a8c2AC93a62d7FBA719fa777026_USDC,0x19c10E1F20df3a8c2AC93a62d7FBA719fa777026_WETH,0x39159C61d549742589E807aDF2Ff2C4Ca8391093_USDC,0x39159C61d549742589E807aDF2Ff2C4Ca8391093_WETH,0xEd15d4De3929AD2Df45D1D954a09f102EAc096d3_USDC,0xEd15d4De3929AD2Df45D1D954a09f102EAc096d3_WETH,...,0x6eB8D6BcccEb84832725DcF792468Dd8ba088449_WETH,0x715134A16acB73c86e81df5542e1CF759eEB6Fc7_USDC,0x715134A16acB73c86e81df5542e1CF759eEB6Fc7_WETH,0xDbcAC7195294bD2f9390ff174f08051262610aE4_USDC,0xDbcAC7195294bD2f9390ff174f08051262610aE4_WETH,0x3Af7A58D54cf014675aF2B7EBc222C3166BF5692_USDC,0x3Af7A58D54cf014675aF2B7EBc222C3166BF5692_WETH,0xB66c84D0D208F2Fe593374D1447fDFB146B2d99d_USDC,0xB66c84D0D208F2Fe593374D1447fDFB146B2d99d_WETH,block
0,1239678.5,0.0,1.684957e+10,157.291695,0.0,212.603006,0.0,61.076919,0.0,0.116451,...,1.612295,0.0,0.687448,0.0,10.469304,0.0,84.872765,0.0,0.316906,18000000
1,1239678.5,0.0,1.601068e+10,160.346405,0.0,212.603006,0.0,61.076919,0.0,0.116451,...,1.612295,0.0,0.687448,0.0,10.469304,0.0,84.872765,0.0,0.316906,18000100
2,1239678.5,0.0,1.601068e+10,160.346405,0.0,212.603006,0.0,61.076919,0.0,0.116451,...,1.612295,0.0,0.687448,0.0,10.469304,0.0,84.872765,0.0,0.316906,18000200
3,1239678.5,0.0,1.889343e+10,148.632997,0.0,212.377981,0.0,61.076919,0.0,0.116451,...,1.612295,0.0,0.687448,0.0,10.469304,0.0,84.872765,0.0,0.316906,18000300
4,1239678.5,0.0,1.889343e+10,151.600760,0.0,212.377981,0.0,61.076919,0.0,0.116451,...,1.612295,0.0,0.687448,0.0,10.469304,0.0,84.872765,0.0,0.316906,18000400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1239678.5,0.0,1.672326e+10,111.167237,0.0,208.488619,0.0,61.076919,0.0,0.116451,...,1.612295,0.0,0.696873,0.0,10.469304,0.0,84.848718,0.0,0.316906,18009500
96,1239678.5,0.0,1.652726e+10,111.167237,0.0,208.488619,0.0,61.076919,0.0,0.116451,...,1.612295,0.0,0.696873,0.0,10.469304,0.0,84.832224,0.0,0.316906,18009600
97,1239678.5,0.0,1.694303e+10,109.657586,0.0,208.488619,0.0,61.076919,0.0,0.116451,...,1.612295,0.0,0.716873,0.0,10.469304,0.0,84.832224,0.0,0.316906,18009700
98,1239678.5,0.0,1.635565e+10,111.793571,0.0,208.488619,0.0,61.076919,0.0,0.116451,...,1.612295,0.0,0.716873,0.0,10.469304,0.0,84.832224,0.0,0.316906,18009800
